In [10]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler


In [3]:
df = pd.read_csv('./processed_errors.csv', header=None, names=['endTime', 'startTime', 'level', 'x','y'])

In [4]:
#bucketing, 500x500. origin is the top left of the map
#these boundaries are the max/min lat/lon in the entire dataset.
#max_lat = 20.108879999999999 
#origin_lat = 18.807134
#max_lon = -98.195071
#origin_lon = -99.662641

#these boundaries are the max/min lat/lon for mexico city only.
max_lat = 19.595892
origin_lat = 19.220471
max_lon = -98.828109
origin_lon = -99.438230

In [6]:
#first, we cut out all the entries in our dataframe which don't fit the desired latitude/longitude.
df = df[(df['y'] < max_lat) & (df['y'] > origin_lat) & (df['x'] < max_lon) & (df['x'] > origin_lon)]

#next, we assign each row an x_bucket or y_bucket based upon its latitude and longitude.
lon_step_size = abs(max_lon - origin_lon)/500
lat_step_size = abs(max_lat - origin_lat)/500
df['x_bucket'] = df['x'].apply(lambda x: abs(x - origin_lon)//lon_step_size)
df['y_bucket'] = df['y'].apply(lambda y: abs(y - origin_lat)//lat_step_size)

/Users/gaoag/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/gaoag/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [13]:
# now, we groupby bucket and sum the level
grouped = df.groupby(['x_bucket', 'y_bucket']).agg({'level':'sum'})
# we normalize values on a 0-100 scale so we have a better notion of intensity
scaler = MinMaxScaler(feature_range=(0,100))
scaler.fit(grouped['level'].reshape(-1, 1))
grouped['level'] = scaler.transform(grouped['level'].reshape(-1, 1))

/Users/gaoag/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """
/Users/gaoag/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/gaoag/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  


In [14]:
# now, we write out to file
grouped.to_csv('./bucketed_intensity.csv')

In [ ]:
# demonstrate how we can reverse lookup
lookup_table = pd.read_csv('./bucketed_intensity.csv')

# let's say you want to lookup an intensity for lat=19.4 lon=-99
def lookup(lookup_table, x, y, origin_lon, origin_lat, lon_step_size, lat_step_size):
    x_bucket = abs(x - origin_lon)//lon_step_size
    y_bucket = abs(y - origin_lat)//lat_step_size
    
    lookup_table


x = -99
y = 19.4
x_bucket = abs(x - origin_lon)//lon_step_size
y_bucket = abs(y - origin_lat)//lat_step_size

lookup_table